In [54]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from tqdm import tqdm
init_notebook_mode(connected=True)

pd.options.display.max_columns = 500
#import pandas_profiling

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    #plt.savefig('lgbm_importances01.png')

In [55]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

var_list = [f for f in train_df.columns if 'var' in f]


In [56]:



train_len = train_df.shape[0]
merged_df = pd.concat([train_df, test_df],sort=False)



In [57]:

# for var in tqdm(var_list):
    
#     merged_df['random_{}'.format(var)] = np.random.normal(merged_df[var].mean(), merged_df[var].std(), 400000).round(4)
#     merged_df['frequency_{}'.format(var)] = merged_df[var].map(merged_df[var].value_counts())
#     merged_df['true_frequency_{}'.format(var)] = merged_df[var].map(merged_df['random_{}'.format(var)].value_counts())
#     merged_df['true_frequency_{}'.format(var)].fillna(0,inplace=True)
#     merged_df['freq_diff_{}'.format(var)] = abs(merged_df['frequency_{}'.format(var)] - merged_df['true_frequency_{}'.format(var)]) 
    
    
    
    
    

In [58]:

# for var in tqdm(var_list):
    
#     random = pd.DataFrame(np.random.normal(merged_df[var].mean(), merged_df[var].std(), 4000000).round(4))
#     random.columns = ['random']

#     #merged_df['frequency_{}'.format(var)] = merged_df[var].map(random['random'].value_counts())
#     merged_df['true_frequency_{}'.format(var)] = merged_df[var].map(random['random'].value_counts())
#     merged_df['true_frequency_{}'.format(var)].fillna(0,inplace=True)
# #    merged_df['freq_diff_{}'.format(var)] = abs(merged_df['frequency_{}'.format(var)] - merged_df['true_frequency_{}'.format(var)]) 
    
    
    
    

In [59]:
for var in tqdm(var_list):
    
    random = pd.DataFrame(np.random.normal(merged_df[var].mean(), merged_df[var].std(), 4000000).round(4))
    random.columns = ['random']
    #random['random'].value_counts
    #random['true_prob'] = random['random']/4000000
    merged_df['frequency_{}'.format(var)] = merged_df[var].map(merged_df[var].value_counts())
    merged_df['prob_{}'.format(var)] = merged_df['frequency_{}'.format(var)]/merged_df.shape[0]
    merged_df['true_prob_{}'.format(var)] = merged_df[var].map(random['random'].value_counts())/4000000
    merged_df['true_prob_{}'.format(var)].fillna(0,inplace=True)
    merged_df['prob_diff_{}'.format(var)] = abs(merged_df['prob_{}'.format(var)] - merged_df['true_prob_{}'.format(var)]) 

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:45<00:00,  1.01it/s]


In [60]:

train_df = merged_df.iloc[:train_len]
test_df = merged_df.iloc[train_len:]

In [66]:
test_df.tail()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,frequency_var_0,prob_var_0,true_prob_var_0,prob_diff_var_0,frequency_var_1,prob_var_1,true_prob_var_1,prob_diff_var_1,frequency_var_2,prob_var_2,true_prob_var_2,prob_diff_var_2,frequency_var_3,prob_var_3,true_prob_var_3,prob_diff_var_3,frequency_var_4,prob_var_4,true_prob_var_4,prob_diff_var_4,frequency_var_5,prob_var_5,true_prob_var_5,prob_diff_var_5,frequency_var_6,prob_var_6,true_prob_var_6,prob_diff_var_6,frequency_var_7,prob_var_7,true_prob_var_7,prob_diff_var_7,frequency_var_8,prob_var_8,true_prob_var_8,prob_diff_var_8,frequency_var_9,prob_var_9,true_prob_var_9,prob_diff_var_9,frequency_var_10,prob_var_10,true_prob_var_10,prob_diff_var_10,frequency_var_11,prob_var_11,true_prob_var_11,prob_diff_var_11,...,true_prob_var_137,prob_diff_var_137,frequency_var_138,prob_var_138,true_prob_var_138,prob_diff_var_138,frequency_var_139,prob_var_139,true_prob_var_139,prob_diff_var_139,frequency_var_140,prob_var_140,true_prob_var_140,prob_diff_var_140,frequency_var_141,prob_var_141,true_prob_var_141,prob_diff_var_141,frequency_var_142,prob_var_142,true_prob_var_142,prob_diff_var_142,frequency_var_143,prob_var_143,true_prob_var_143,prob_diff_var_143,frequency_var_144,prob_var_144,true_prob_var_144,prob_diff_var_144,frequency_var_145,prob_var_145,true_prob_var_145,prob_diff_var_145,frequency_var_146,prob_var_146,true_prob_var_146,prob_diff_var_146,frequency_var_147,prob_var_147,true_prob_var_147,prob_diff_var_147,frequency_var_148,prob_var_148,true_prob_var_148,prob_diff_var_148,frequency_var_149,prob_var_149,true_prob_var_149,prob_diff_var_149,frequency_var_150,prob_var_150,true_prob_var_150,prob_diff_var_150,frequency_var_151,prob_var_151,true_prob_var_151,prob_diff_var_151,frequency_var_152,prob_var_152,true_prob_var_152,prob_diff_var_152,frequency_var_153,prob_var_153,true_prob_var_153,prob_diff_var_153,frequency_var_154,prob_var_154,true_prob_var_154,prob_diff_var_154,frequency_var_155,prob_var_155,true_prob_var_155,prob_diff_var_155,frequency_var_156,prob_var_156,true_prob_var_156,prob_diff_var_156,frequency_var_157,prob_var_157,true_prob_var_157,prob_diff_var_157,frequency_var_158,prob_var_158,true_prob_var_158,prob_diff_var_158,frequency_var_159,prob_var_159,true_prob_var_159,prob_diff_var_159,frequency_var_160,prob_var_160,true_prob_var_160,prob_diff_var_160,frequency_var_161,prob_var_161,true_prob_var_161,prob_diff_var_161,frequency_var_162,prob_var_162,true_prob_var_162,prob_diff_var_162,frequency_var_163,prob_var_163,true_prob_var_163,prob_diff_var_

In [67]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = 'random_prob_diff_10_times'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: random_prob_diff_10_times_5_folds


In [68]:
test_df.columns

Index(['ID_code', 'target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4',
       'var_5', 'var_6', 'var_7',
       ...
       'true_prob_var_197', 'prob_diff_var_197', 'frequency_var_198',
       'prob_var_198', 'true_prob_var_198', 'prob_diff_var_198',
       'frequency_var_199', 'prob_var_199', 'true_prob_var_199',
       'prob_diff_var_199'],
      dtype='object', length=1002)

In [69]:

exclusion = ['ID_code', 'target'] 

for var in tqdm(var_list):
    exclusion.append('frequency_{}'.format(var))
    exclusion.append('prob_{}'.format(var))
    
feats = [c for c in train_df.columns if c not in exclusion]



100%|████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 200301.05it/s]


In [70]:
feats
    

['var_0',
 'var_1',
 'var_2',
 'var_3',
 'var_4',
 'var_5',
 'var_6',
 'var_7',
 'var_8',
 'var_9',
 'var_10',
 'var_11',
 'var_12',
 'var_13',
 'var_14',
 'var_15',
 'var_16',
 'var_17',
 'var_18',
 'var_19',
 'var_20',
 'var_21',
 'var_22',
 'var_23',
 'var_24',
 'var_25',
 'var_26',
 'var_27',
 'var_28',
 'var_29',
 'var_30',
 'var_31',
 'var_32',
 'var_33',
 'var_34',
 'var_35',
 'var_36',
 'var_37',
 'var_38',
 'var_39',
 'var_40',
 'var_41',
 'var_42',
 'var_43',
 'var_44',
 'var_45',
 'var_46',
 'var_47',
 'var_48',
 'var_49',
 'var_50',
 'var_51',
 'var_52',
 'var_53',
 'var_54',
 'var_55',
 'var_56',
 'var_57',
 'var_58',
 'var_59',
 'var_60',
 'var_61',
 'var_62',
 'var_63',
 'var_64',
 'var_65',
 'var_66',
 'var_67',
 'var_68',
 'var_69',
 'var_70',
 'var_71',
 'var_72',
 'var_73',
 'var_74',
 'var_75',
 'var_76',
 'var_77',
 'var_78',
 'var_79',
 'var_80',
 'var_81',
 'var_82',
 'var_83',
 'var_84',
 'var_85',
 'var_86',
 'var_87',
 'var_88',
 'var_89',
 'var_90',
 'var_91'

In [ ]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 5, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]
    
    
#     for col_index, new_col_name in enumerate(tqdm(new_cols)):
#         trn_x[new_col_name] = trn_x.iloc[:,[col_index]].apply(lambda x: x.map(trn_x.iloc[:,col_index].value_counts()))
#         X_test[new_col_name] = X_test.iloc[:,[col_index]].apply(lambda x: x.map(trn_x.iloc[:,col_index].value_counts()))
#         val_x[new_col_name] = val_x.iloc[:,[col_index]].apply(lambda x: x.map(trn_x.iloc[:,col_index].value_counts()))
        
    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=30000,
                     early_stopping_rounds=50,
                     verbose_eval=500)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
   # print('getting feature importance')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

display_importances(feature_importance_df)


Current Fold: 1
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.893892	valid_1's auc: 0.872173
[1000]	training's auc: 0.915937	valid_1's auc: 0.890005
[1500]	training's auc: 0.926682	valid_1's auc: 0.897489
[2000]	training's auc: 0.934816	valid_1's auc: 0.900343
Early stopping, best iteration is:
[2190]	training's auc: 0.937902	valid_1's auc: 0.901104
AUC = 0.9011039823256635
Current Fold: 2
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.894445	valid_1's auc: 0.873207
[1000]	training's auc: 0.916512	valid_1's auc: 0.890967
[1500]	training's auc: 0.927123	valid_1's auc: 0.898092
[2000]	training's auc: 0.935127	valid_1's auc: 0.900571
Early stopping, best iteration is:
[2161]	training's auc: 0.937781	valid_1's auc: 0.900897
AUC = 0.9008973987238427
Current Fold: 3
Training until validation scores don't improve for 50 rounds.
[500]	training's auc: 0.893887	valid_1's auc: 0.878948
[1000]	training's auc: 0.915805	v

In [33]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [34]:


shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.ipynb'.format(model_name, str(roc_score)))


'../models/frequency_diff_to_random_5_folds_0.9008732896680773.ipynb'